In [3]:
!pip install pyspark sparknlp pandas

     |████████████████████████████████| 317.0 MB 18 kB/s  eta 0:00:011    |██████▊                         | 66.5 MB 4.6 MB/s eta 0:00:55     |██████▊                         | 66.9 MB 4.6 MB/s eta 0:00:55     |█████████████▍                  | 132.8 MB 4.0 MB/s eta 0:00:46     |█████████████▉                  | 136.9 MB 2.7 MB/s eta 0:01:06     |██████████████▊                 | 145.8 MB 5.0 MB/s eta 0:00:35     |████████████████▊               | 165.5 MB 5.8 MB/s eta 0:00:26     |████████████████▉               | 167.2 MB 7.5 MB/s eta 0:00:20     |█████████████████████████▏      | 248.8 MB 4.7 MB/s eta 0:00:15 |██████████████████████████▉     | 266.1 MB 4.7 MB/s eta 0:00:11     |███████████████████████████▍    | 270.9 MB 1.0 MB/s eta 0:00:46     |███████████████████████████▌    | 272.6 MB 1.0 MB/s eta 0:00:44     |████████████████████████████    | 276.6 MB 6.5 MB/s eta 0:00:07
     |████████████████████████████████| 200 kB 3.1 MB/s eta 0:00:01
Using legacy 'setup.py install' for pysp

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType, DataType,ArrayType
from pyspark.sql.functions import udf, struct
from pyspark.ml import Pipeline
from IPython.core.display import display, HTML
import re


/var/folders/fv/z4h0lznx44n3wybc34chcwhr0000gn/T/ipykernel_95845/3309149586.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [7]:
import sparknlp

from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *

spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

:: loading settings :: url = jar:file:/Users/isabeladias/.pyenv/versions/3.9.6/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/isabeladias/.ivy2/cache
The jars for the packages stored in: /Users/isabeladias/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f073322f-79e0-456f-9354-a94835d40390;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.4.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.5

Spark NLP version 5.4.0
Apache Spark version: 3.5.1


In [9]:
stopwords = StopWordsCleaner().getStopWords()

document = DocumentAssembler() \
            .setInputCol("text") \
            .setOutputCol("document")

sentenceDetector = SentenceDetector() \
            .setInputCols("document") \
            .setOutputCol("sentence")

token = Tokenizer() \
            .setInputCols("sentence") \
            .setOutputCol("token") \
            .setContextChars(["(", ")", "?", "!", ".", ","])

keywords = YakeKeywordExtraction() \
            .setInputCols("token") \
            .setOutputCol("keywords") \
            .setMinNGrams(1) \
            .setMaxNGrams(3)\
            .setNKeywords(20)\
            .setStopWords(stopwords)

yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])

empty_df = spark.createDataFrame([['']]).toDF("text")

yake_Model = yake_pipeline.fit(empty_df)



In [12]:
# LightPipeline

light_model = LightPipeline(yake_Model)

text = 'Insects are one of the most abundant and diverse forms of life on the planet, representing about 80% of all known animal species. They can be found in nearly every environment, from the depths of caves to the peaks of the highest mountains, and they play crucial roles in ecosystems as pollinators, decomposers, and as fundamental parts of food chains. The class Insecta is divided into various groups, including butterflies, bees, ants, beetles, flies, and many others. Each group has unique characteristics that allow them to survive and thrive in their respective habitats. For example, butterflies are known for their colorful wings and migration patterns, while ants are notorious for their highly organized and complex societies. Insects have a segmented body divided into three main parts: head, thorax, and abdomen. The head usually has antennae, compound eyes, and mouthparts adapted to different types of feeding. The thorax is where the wings and legs are located, allowing for efficient locomotion. The abdomen houses many of the internal organs, such as the digestive and reproductive systems. The ability to fly is one of the most significant evolutionary innovations of insects. Wings have allowed many insects to colonize new habitats, escape predators, and seek food more efficiently. Bees, for example, are essential pollinators for many flowering plants, including several agricultural crops important to humanity. Insects also play a vital role in the decomposition of organic matter. Beetles, flies, and ants help break down plant and animal remains, recycling nutrients back into the soil. This process is fundamental for the health of ecosystems, as it maintains the nutrient cycle and promotes the growth of new plants. However, not all insects are beneficial to humans. Some are considered pests, such as locusts, which can devastate crops, and mosquitoes, which are vectors of diseases like malaria, dengue, and Zika. Controlling these pests is an ongoing challenge and requires a balance between protecting human health and preserving natural ecosystems. The diversity of forms, colors, and behaviors of insects is impressive. Some, like the praying mantis, are masters of camouflage, mimicking leaves and branches to hide from predators or ambush prey. Others, like fireflies, produce their own light through a process called bioluminescence, used to attract mates or disorient predators. The study of insects, known as entomology, has a long history and continues to be a dynamic field of research. Entomologists investigate everything from the ecology and evolution of insects to their interactions with other organisms and their impact on human environments. This research has practical applications in areas such as agriculture, medicine, and conservation. Recently, concern about the decline of insect populations has been growing. Factors such as habitat loss, pesticide use, pollution, and climate change are contributing to the reduction of many insect species populations. This decline can have severe consequences for ecosystems and food production, given the essential role of insects in pollination and decomposition. To ensure the conservation of insects, it is crucial to promote sustainable and conscious practices that protect their natural habitats and minimize the negative impacts of human activities. Education and awareness about the importance of insects can help foster greater appreciation and protection of these small but vital inhabitants of our planet.'


light_result = light_model.fullAnnotate(text)[0]

[(s.metadata['sentence'], s.result) for s in light_result['sentence']]

[('0',
  'Insects are one of the most abundant and diverse forms of life on the planet, representing about 80% of all known animal species.'),
 ('1',
  'They can be found in nearly every environment, from the depths of caves to the peaks of the highest mountains, and they play crucial roles in ecosystems as pollinators, decomposers, and as fundamental parts of food chains.'),
 ('2',
  'The class Insecta is divided into various groups, including butterflies, bees, ants, beetles, flies, and many others.'),
 ('3',
  'Each group has unique characteristics that allow them to survive and thrive in their respective habitats.'),
 ('4',
  'For example, butterflies are known for their colorful wings and migration patterns, while ants are notorious for their highly organized and complex societies.'),
 ('5',
  'Insects have a segmented body divided into three main parts: head, thorax, and abdomen.'),
 ('6',
  'The head usually has antennae, compound eyes, and mouthparts adapted to different types 

In [15]:
import pandas as pd

keys_df = pd.DataFrame([(k.result, k.begin, k.end, k.metadata['score'],  k.metadata['sentence']) for k in light_result['keywords']],
                       columns = ['keywords','begin','end','score','sentence'])
keys_df['score'] = keys_df['score'].astype(float)

# ordered by relevance
keys_df.sort_values(['sentence','score']).head(30)

,keywords,begin,end,score,sentence
0,insects,0,6,0.058036,0
2,known,108,112,0.268544,0
1,one,12,14,0.424803,0
3,animal,114,119,0.435028,0
5,ecosystems,272,281,0.206005,1
7,food,340,343,0.281761,1
4,play,250,253,0.435028,1
6,pollinators,286,296,0.435028,1
29,insects,1218,1224,0.058036,10
28,many,1213,1216,0.156446,10


In [19]:
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

df = spark.read\
                .option("header", "true")\
                .csv("pubmed_sample_text_small.csv")\

df.show(truncate=50)


+--------------------------------------------------+
|                                              text|
+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
|BACKGROUND: At present, it is one of the most i...|
|OBJECTIVE: To investigate the relationship betw...|
|Combined EEG/fMRI recording has been used to lo...|
|Kohlschutter syndrome is a rare neurodegenerati...|
|Statistical analysis of neuroimages is commonly...|
|The synthetic DOX-LNA conjugate was characteriz...|
|Our objective was to compare three different me...|
|We conducted a phase II study to assess the eff...|
|"""Monomeric sarcosine oxidase (MSOX) is a flav...|
|We presented the tachinid fly Exorista japonica...|
|The literature dealing with the water conductin...|
|A novel approach to synthesize chitosan-O-isopr...|
|An HPLC-ESI-MS-MS method has been developed for...|
|The localizing and lateralizing values of eye a...|
|OBJECTIVE: To evaluate the effectiveness and 

In [20]:
result = yake_pipeline.fit(df).transform(df)
result = result.withColumn('unique_keywords', F.array_distinct("keywords.result"))
def highlight(text, keywords):
    for k in keywords:
        text = (re.sub(r'(\b%s\b)'%k, r'<span style="background-color: yellow;">\1</span>', text, flags=re.IGNORECASE))
    return text
highlight_udf = udf(highlight, StringType())
result = result.withColumn("highlighted_keywords",highlight_udf('text','unique_keywords'))
for r in result.select("highlighted_keywords").limit(20).collect():
    display(HTML(r.highlighted_keywords))
    print("\n\n")

